SEURAT
v5.2.0
SEURAT is a R-based tool for single cell genomics analysis, particularly for the clustering analysis.  

The SEURAT package was also resistant against running. Still I got it.
See "seurat_installation_script.r"

SEURAT is also required to read out feature-barcode-matrices, outputs of cellranger count run. 
First of all, the raw feature-barcode-matrix is read with Read10X command of Seurat. 

In [ ]:
library(Biostrings)
library(Seurat)
getwd()
setwd("E:/scTriH2/SEURAT/extracted/1.cellRangerCount/")
raw_matrix_SRR24886407 <- Read10X(data.dir = "SRR24886407/outs/raw_feature_bc_matrix/")
print(head(raw_matrix_SRR24886407))
ncol(raw_matrix_SRR24886407)
nrow(raw_matrix_SRR24886407)

#### 1318799 barcodes (total cell count in the sample)
#### 1252 features (1252 mappable features)

First, check whether the total mapped gene count is correct (see web_summary_SRR*.html)
Then, the matrix is filterd. 
Filter out every feature with less than 2 hits per cell. (min.features)
    - 1 inclusions can be contamination or coincident inclusion of feeding bacteria
    - Features with higher than 2 inclusions is assumed as endosymbiotic genes
    - Due to further filtering (e.g. by UMI count), the quality control will be improved
    - Here, low-quality cells or empty droplets (barcodes, columns) are removed but not genes (features, row)

Also, a gene must be contained across multiple cells: minimum of 2 cells must contain a gene (min.cell)
    - Due to low detection level, the minimum cell number is 2 instead of 3 or more. 
    - Here, lowly expressed genes or background noise (features, row) is removed


In [ ]:
raw_matrix_SRR24886407 <- raw_matrix_SRR24886407[rowSums(raw_matrix_SRR24886407>0)>= 1, ]
nrow(raw_matrix_SRR24886407)
raw_matrix_SRR24886407 <- raw_matrix_SRR24886407[rowSums(raw_matrix_SRR24886407>0)>= 2, ]
nrow(raw_matrix_SRR24886407)

raw_SRR24886407 <- CreateSeuratObject(counts = raw_matrix_SRR24886407, project = "scTriH2", min.features = 2, min.cells = 2)
nrow(raw_SRR24886407[["RNA"]]$counts)
ncol(raw_SRR24886407[["RNA"]]$counts)

##### filtering by minium expressed gene count = 1, 355 features left, aligns with the summary
##### filtering by minium expressed gene count = 2, 179 features left
##### after filtering by gene count per cell (min.features=2) and cell per gene (min.cells) , only 71 features and 1646 barcodes remain. 
##### if only last filering step undertaken (CreateSeuratObject), then other numbers: 71 features but 1665 barcodes remaining. 

##### From now on, the underscores are replaced by dashes due to the error: "Warning: Feature names cannot have underscores ('_'), replacing with dashes ('-')"

### WeirdThing: When I prefilter it with rowSums >=2 and then create seurat object with the metrics, the result is different than only creating seurat object with the same metrics (min.cells=2, min.features=2)

Small QC-session:
Check and visualize with violin plot:
- UMI (mapped bacterial read) count per cell (nCount_RNA)
    - This should be higher than 
- Number of expressed genes per cell (nFeature_RNA)
- Percentage of reads assigned to bacterial genes (percent_bact) 
    - assessed later with the data of metacell.rda

Then, filtered accordingly

    class, cluster, foreign, KernSmooth, MASS,
    Matrix, nlme, nnet, rpart, spatial, survival
Old packages: 'curl', 'GenomeInfoDb', 'parallelly',
  'Rcpp', 'rlang', 'sp', 'spam'